In [1]:
! pip install pynini

  Using cached pynini-2.1.6.tar.gz (789 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pynini


  error: subprocess-exited-with-error
  
  × Building wheel for pynini (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [64 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\pynini
      copying pynini\__init__.py -> build\lib.win-amd64-cpython-311\pynini
      creating build\lib.win-amd64-cpython-311\pywrapfst
      copying pywrapfst\__init__.py -> build\lib.win-amd64-cpython-311\pywrapfst
      creating build\lib.win-amd64-cpython-311\pynini\examples
      copying pynini\examples\case.py -> build\lib.win-amd64-cpython-311\pynini\examples
      copying pynini\examples\chatspeak.py -> build\lib.win-amd64-cpython-311\pynini\examples
      copying pynini\examples\chatspeak_model.py -> build\lib.win-amd64-cpython-311\pynini\examples
      copying pynini\examples\dates.py -> build\lib.win-amd64-cpython-311\pynini\examples
  

In [ ]:
import pynini as pn

In [ ]:
fsa1 = (pn.closure(pn.accep("a")) + pn.accep("b") + pn.accep("b") + pn.closure(pn.accep("c"), 1))
fsa1

In [ ]:
ascii_table = pn.SymbolTable()
ascii_table.add_symbol("<eps>", 0)
for i in range(12, 128):
  ascii_table.add_symbol(chr(i), i)

fsa1.set_input_symbols(ascii_table)

In [ ]:
def draw(fst, symbol_table=ascii_table):
  fst.set_input_symbols(symbol_table)
  fst.set_output_symbols(symbol_table)
  return fst

draw(fsa1)

In [2]:
draw(pn.optimize(pn.closure(pn.accep("a")) + pn.accep("b") + pn.accep("b") + pn.closure(pn.accep("c"), 1)))

NameError: name 'draw' is not defined

In [ ]:
import pynini.lib.rewrite as pn_rewrite

pn_rewrite.matches("abbc", "abbc", fsa1)

In [ ]:
fsa1 = pn.optimize(pn.closure(pn.accep("a")) + pn.accep("bb") + pn.closure(pn.accep("c"), 1))
draw(fsa1)

In [ ]:
draw(pn.optimize((pn.accep("a").plus)))

In [ ]:
a_1_to_9 = pn.union("one", "two", "three", "four", "five", "six", "seven", "eight", "nine")
draw(a_1_to_9)

In [ ]:
a_1_to_9.optimize()

In [ ]:
draw("one" @ a_1_to_9)
draw("foo" @ a_1_to_9)

In [ ]:
a_1_to_9_dollars = (a_1_to_9 + " dollars")
draw("two dollars" @ a_1_to_9_dollars)

In [ ]:
a_2_to_9 = pn.union("two", "three", "four", "five", "six", "seven", "eight", "nine")
dollars = pn.union("one dollar", a_2_to_9 + " dollars")

draw("one dollar" @ dollars)
draw("one dollars" @ dollars)
draw("two dollars" @ dollars)

In [ ]:
cents = pn.union("one cent", a_2_to_9 + " cents")
money = pn.union(dollars, cents, dollars + " " + cents)
draw("two cents" @ money)

In [ ]:
draw("two dollars one cent" @ money)
draw(money)

In [ ]:
a_2_to_9_words = pn.union("[two]", "[three]", "[four]", "[five]", "[six]", "[seven]", "[eight]", "[nine]")
dollars_words = pn.union("[one][dollar]", a_2_to_9_words + "[dollars]")
cents_words = pn.union("[one][cent]", a_2_to_9_words + "[cents]")
money_words = pn.union(dollars_words, cents_words, dollars_words + cents_words).optimize()
money_words.set_input_symbols(pn.generated_symbols())

In [ ]:
draw("[one][dollar]" @ money_words, pn.generated_symbols())

In [ ]:
pn.randgen(money_words)

In [ ]:
pn.randgen(money_words, 3)

Construct a FSA that accepts English (or Estonian) numbers from 0 to 99, given as words, e.g. `[twenty] [one]`. Try to be economical, don't just list all possible 100 variants

In [ ]:
from pynini.lib import pynutil

def construct_fsa():
  single_digit = pn.union("zero", "one", "two", "three", "four", "five", "six", "seven", "eight", "nine")
  teens = pn.union("ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen", "sixteen", "seventeen", "eighteen", "nineteen")
  tens = pn.union("twenty", "thirty", "forty", "fifty", "sixty", "seventy", "eighty", "ninety")
  compound = tens + pynutil.insert(" ") + single_digit
  fsa = single_digit | teens| tens | compound

  return fsa

In [ ]:
fsa = construct_fsa()
draw(fsa)

In [ ]:
reg_noun = pn.union("dog", "cat")
irreg_noun_sg = pn.union("mouse", "sheep")
irreg_noun_pl = pn.union("mice", "sheep")
nouns = pn.union(reg_noun, reg_noun + "s", irreg_noun_sg, irreg_noun_pl).optimize()
draw(nouns)

In [ ]:
draw("dogs" @ nouns)
draw("mouses" @ nouns)
pn.randgen(nouns)
pn.randgen(nouns).string()

Create a FST that accepts Estonian nouns with at least 3 words and 3 inflections. The words are karu, laud, vahvel, and the inflections are nimetav (karu - laud - vahvel), omastav (karu - laua - vahvli) and kaasaütlev (karuga - lauaga - vahvliga). Try to reuse parts omastav when constructing kaasaütlev, instead of just listing all nine varints of the words. That is, it should be easy to extend this FSA to understand new inflections.

In [ ]:
words = {
    "karu": ("karu", "karu", "karuga"),
    "laud": ("laud", "laua", "lauaga"),
    "vahvel": ("vahvel", "vahvli", "vahvliga")
}

karu = pn.union("karu", "karu", "karuga")
laud = pn.union("laud", "laua", "lauaga")
vahvel = pn.union("vahvel", "vahvli", "vahvliga")

words_fst = pn.union(karu, laud, vahvel)

def create_inflections(word: str, nom: str, gen: str, com: str):
    nom_fst = pn.union(nom)
    gen_fst = pn.union(gen)
    com_fst = pynutil.insert(com) + pn.union("ga")

    return nom_fst | gen_fst | com_fst

karu_fst = create_inflections("karu", "karu", "karu", "karuga")
laud_fst = create_inflections("laud", "laud", "laua", "lauaga")
vahvel_fst = create_inflections("vahvel", "vahvel", "vahvli", "vahvliga")

combined_fst = karu_fst | laud_fst | vahvel_fst

combined_fst.optimize()

draw(combined_fst)

Transducers

In [ ]:
draw(pn.cross("a", "b"))

fst0 = (pn.accep("aa") + pn.cross("b", "c") + pn.accep("dd")).optimize()
draw(fst0)

In [ ]:
fst1 = (pn.cross("aa", "b").closure() + pn.union(pn.cross("b", "a"), pn.accep("b")) + pn.cross("b", "").closure()).optimize()
draw(fst1)

In [ ]:
my_fst = (pn.cross("aaa", "bcb")) + pn.union("a", "bb") + pn.union(pn.cross("cc", "aba") + pn.cross("fa", "af").closure())
my_fst.optimize()

draw(my_fst)

In [ ]:
draw("b" @ fst1)

In [ ]:
list(("b" @ fst1).paths().ostrings())

def all_paths(fst):
    return list(fst.paths().ostrings())

all_paths("b" @ fst1)

In [ ]:
draw("aabbbb" @ fst1)
all_paths("aabbbb" @ fst1)

In [ ]:
fst1_invert = pn.invert(fst1)
fst1_invert

In [ ]:
draw("a" @ fst1_invert)
all_paths("a" @ fst1_invert)

In [ ]:
list((pn.shortestpath("a" @ fst1_invert, nshortest=10)).paths().ostrings())

In [ ]:
def top10_paths(fst):
    return list(pn.shortestpath(fst, nshortest=10).paths().ostrings())

top10_paths("a" @ fst1_invert)

In [ ]:
stems = pn.union("ratsu", "kõne", "male")
nominative = stems + pn.cross("", " /nom/")

top10_paths("ratsu" @ nominative)

In [ ]:
genitive = stems + pn.cross("", " /gen/")
partitive = stems + pn.cross("t", " /p/")
illative = stems + pn.cross("sse", " /ill/")

wordform_to_analysis = pn.union(nominative, genitive, partitive, illative)

top10_paths("ratsu" @ wordform_to_analysis)

top10_paths("kõnesse" @ wordform_to_analysis)[0]

analysis_to_wordform = pn.invert(wordform_to_analysis)
top10_paths("male /ill/" @ analysis_to_wordform)

In [ ]:
rewrite_fst = pn.cdrewrite(pn.cross("b", "c"), "a", "d", pn.union(*"abcd").closure()).optimize()
draw(rewrite_fst)

In [ ]:
top10_paths("abd" @ rewrite_fst)
top10_paths("abbabdc" @ rewrite_fst)
top10_paths("b" @ rewrite_fst)

rewrite_fst2 = pn.cdrewrite(pn.cross("b", "c"), "a", "", pn.union(*"abcd").closure()).optimize()
top10_paths("abbbaa" @ rewrite_fst2)

In [ ]:
rewrite_fst3 = pn.cdrewrite(pn.cross("b", "c"), "a" + pn.union(*"abcd").closure(), "", pn.union(*"abcd").closure()).optimize()
top10_paths("ab" @ rewrite_fst3)

top10_paths("cccb" @ rewrite_fst3)
top10_paths("acccb" @ rewrite_fst3)

In [ ]:
words = ["number", "vahvel", "noobel"]
end_analyzer = pn.string_map(words) + pn.union(pn.cross("^it", " /part/"), pn.cross("^isse", " /ill/"), pn.cross("^i", " /gen/"), pn.cross("", " /nom/")).optimize()

top10_paths("number^i" @ end_analyzer)

In [ ]:
sigma_star = pn.union(*"abdefghjiklmnoprstuvõäöüxy^").closure()
stem_change = pn.cdrewrite(pn.union(pn.cross("li", "el^i"), pn.cross("ri", "er^i")), "", pn.union("", "s", "sse", "ga") + "[EOS]", sigma_star).optimize()

top10_paths("numbri" @ stem_change)

In [ ]:
top10_paths("ribambri" @ stem_change)

In [ ]:
wordform_to_analysis2 = stem_change @ end_analyzer

top10_paths("numbri" @ wordform_to_analysis2)

In [ ]:
analysis_to_word2 = pn.invert(wordform_to_analysis2)
top10_paths("number /gen/" @ analysis_to_word2)

In [ ]:
valid_chars = pn.union(*"abdefghjiklmnoprstuvõäöüxy").closure().optimize()
wordform_to_analysis2 = (valid_chars @ stem_change @ end_analyzer)
top10_paths("numbri" @ wordform_to_analysis2)

In [ ]:
top10_paths("number^i" @ wordform_to_analysis2)

In [ ]:
analysis_to_word2 = pn.invert(wordform_to_analysis2)
top10_paths("number /gen/" @ analysis_to_word2)

In [ ]:
composite_wordform_to_analysis = pn.union(wordform_to_analysis, wordform_to_analysis2)

In [ ]:
top10_paths("numbri" @ composite_wordform_to_analysis)
top10_paths("ratsu" @ composite_wordform_to_analysis)

In [ ]:
for w in (top10_paths(pn.randgen(composite_wordform_to_analysis, 5))): print(w)

In [ ]:
one_character = pn.union(*"abdefghjiklmnoprstuvõäöüxy")
spacer = (one_character +  pn.cross("", " ")).closure() + one_character

("test" @ spacer).string()